# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
source /path/to/your/venv/bin/activate


In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

    # move data to GPU


Using device: cuda


In [6]:
import sys

project_root = "/Users/heydari/Documents/TEMA-FHHI-PY/FHHI-XAI/"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [7]:
import torch
import torchvision.transforms as transforms

# Add the parent directory to the Python path - bad practice, but it's just for the example
import sys

import sys
project_root = "/home/heydari/FHHI-XAI/"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

#import sys
#sys.path.append("/Users/heydari/Documents/TEMA-FHHI-PY/FHHI-XAI/")

from src.glocal_analysis import run_analysis
from src.plot_crp_explanations import plot_explanations
from src.datasets.person_car_dataset import PersonCarDataset
from LCRP.models import get_model
import LCRP.models.yolov6 as yolov6

In [7]:
import torch

#device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
device

'cpu'

# Loading dataset and model

In [11]:
dtype = torch.float32
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Resize((1280, 1280)),
    transforms.Lambda(lambda x: x.to(dtype)), 
])
import sys
project_root = "/home/heydari/FHHI-XAI/"
if project_root not in sys.path:
    sys.path.insert(0, project_root)


# Load dataset
root_dir = "/home/heydari/FHHI-XAI/src/datasets/data/person_car_detection_data/Arthal/"
dataset = PersonCarDataset(root_dir=root_dir, split="train", transform=transform)

model_name = "yolov6s6"
ckpt_path = "/home/heydari/FHHI-XAI/src/models/best_v6s6_ckpt.pt"

# Loading unet with path to checkpoint
model = get_model(model_name=model_name, classes=2, ckpt_path=ckpt_path, device=device, dtype=dtype)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

: 

In [ ]:
for idx in range(100):
    img, label = dataset[idx]  # Load image and label from dataset
    img = img.unsqueeze(0).to(device)  # Add batch dimension and move to GPU

    scores, boxes = model.predict_with_boxes(img)  # Run inference

    num_boxes = boxes.shape[0] if boxes is not None else 0  # Safe check

    print(f"Image {idx} → Detected {num_boxes} boxes")

In [ ]:
img = img.to(device)


In [ ]:
device

In [ ]:
model.to(device)


In [ ]:
boxes.shape

In [ ]:
scores.argmax(dim=2).shape

In [ ]:
boxes.shape

In [ ]:
# Show the image
import matplotlib.pyplot as plt
plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
out[1].shape

# Running analysis and plotting results

In [ ]:
# This folder contains results of the glocal analysis.

output_dir = "../output/crp/yolo_person_car"

In [ ]:
# Only run this if needed, takes a long time (79 on cpu on my laptop).
# run_analysis(model_name, model, dataset, output_dir="output/crp/yolo_person_car")

In [ ]:
import torch
import copy
import logging
import random
import pandas as pd
import re

# === Setup ===
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

n_samples = 100
dataset_size = len(dataset)
n_concepts = 3
class_id = 1
model_name = "yolov6s6"

yolo_layers = [
    "module.backbone.stem.rbr_dense.conv",
    "module.backbone.ERBlock_2.0.rbr_dense.conv",
    "module.backbone.ERBlock_3.1.block.0.rbr_dense.conv",
    "module.backbone.ERBlock_4.1.block.1.rbr_1x1.conv",
    "module.neck.Rep_p3.conv1.rbr_1x1.conv",
    "module.neck.Rep_p5.block.2.rbr_dense.conv"
]

log_file_path = "../examples/log_yolo_test.txt"

# === Configure Logging ===
logging.basicConfig(filename=log_file_path, level=logging.ERROR, force=True)

# === Attribution Loop ===
sample_indices = random.sample(range(dataset_size), n_samples)

for i, sample_idx in enumerate(sample_indices):
    layer_idx = i % len(yolo_layers)
    layer = yolo_layers[layer_idx]

    image_tensor, _ = dataset[sample_idx]
    image_tensor = image_tensor.to(device)  #  Move to same device as model

    # Log sample & layer before attribution
    logging.error(f"SAMPLE {sample_idx}, LAYER {layer}")

    attribution = ATTRIBUTORS[model_name](model)
    composite = COMPOSITES[model_name](canonizers=[CANONIZERS[model_name]()])
    condition = [{"y": class_id}]

    input_tensor = copy.deepcopy(image_tensor).unsqueeze(0).requires_grad_()

    #  Ensure input_tensor is on correct device (defensive check)
    input_tensor = input_tensor.to(device)

    attr = attribution(
        input_tensor,
        condition,
        composite,
        record_layer=[layer],
        init_rel=1
    )

print("Attribution complete, logs saved.")

# === Parsing Logs & Building Tables ===
with open(log_file_path, 'r') as f:
    logs = f.readlines()

results = []
sample_id = None
layer_name = None

for line in logs:
    sample_match = re.match(r'.*SAMPLE (\d+), LAYER (.+)', line)
    if sample_match:
        sample_id = int(sample_match.group(1))
        layer_name = sample_match.group(2)
    time_match = re.search(r'Prediction time: ([\d\.]+), Backward time: ([\d\.]+), Full attribution time: ([\d\.]+)', line)
    if time_match and sample_id is not None:
        results.append({
            "Sample ID": sample_id,
            "Layer": layer_name,
            "Prediction Time (s)": float(time_match.group(1)),
            "Backward Time (s)": float(time_match.group(2)),
            "Full Attribution Time (s)": float(time_match.group(3))
        })
        sample_id, layer_name = None, None

# === Save Detailed Sample Results ===
df = pd.DataFrame(results)
df = df.sort_values(by="Sample ID")
df.to_csv("detailed_sample_times.csv", index=False)

# === Compute & Save Global Summary Table ===
avg_pred_time = df["Prediction Time (s)"].mean()
global_lcrp_time = df["Full Attribution Time (s)"].sum()
attr_single = df["Backward Time (s)"].mean()
allowed_time = attr_single * dataset_size

summary_table = pd.DataFrame({
    "Metric": [
        "Model Prediction (average per sample)",
        "Global (LCRP) Total Time",
        "Attribution Time per Sample",
        "Allowed Time for Global Explanation",
        "Number of Data Points"
    ],
    "Value": [
        f"{avg_pred_time:.4f} s",
        f"{global_lcrp_time:.2f} s",
        f"{attr_single:.4f} s",
        f"{allowed_time:.2f} s",
        f"{dataset_size}"
    ]
})

summary_table.to_csv("KPI_explanation_summary.csv", index=False)

# === Display Output ===
print("\n==== Global Explanation Summary ====")
print(summary_table.to_string(index=False))
print("Results saved to 'detailed_sample_times.csv' and 'KPI_explanation_summary.csv'")


In [ ]:
import pandas as pd

# Load the detailed sample times (produced by the main script)
df = pd.read_csv("detailed_sample_times.csv")

# Calculate the Backward / Prediction Time Ratio per sample
df["Backward/Prediction Ratio"] = df["Backward Time (s)"] / df["Prediction Time (s)"]
avg_ratio = df["Backward/Prediction Ratio"].mean()

print(f"Average Backward Time / Prediction Time Ratio: {avg_ratio:.2f}")

# Load summary table to append the new metric
summary_table = pd.read_csv("global_explanation_summary.csv")

# Append the ratio to the summary table
summary_table = pd.concat([
    summary_table,
    pd.DataFrame({
        "Metric": ["Average Backward/Prediction Time Ratio"],
        "Value": [f"{avg_ratio:.2f}"]
    })
], ignore_index=True)

# Save updated summary
summary_table.to_csv("global_explanation_summary.csv", index=False)

print("Summary table updated with ratio.")


In [ ]:
# Load detailed per-sample times
df = pd.read_csv("detailed_sample_times.csv")

# Calculate Backward / Prediction ratio per sample
df["Backward/Prediction Ratio"] = df["Backward Time (s)"] / df["Prediction Time (s)"]

# Calculate overall average ratio across all samples
avg_ratio = df["Backward/Prediction Ratio"].mean()

# Group by layer to compute average ratio per layer
layer_avg_ratios = df.groupby("Layer")["Backward/Prediction Ratio"].mean()

# Find the layer with the minimum average ratio
min_layer = layer_avg_ratios.idxmin()
min_ratio = layer_avg_ratios.min()

# === Final Output ===
print("Average Backward Time / Prediction Time Ratio (All Samples):")
print(f"{avg_ratio:.2f}")

print("Layer with Minimum Average Backward/Prediction Time Ratio:")
print(f"Layer: {min_layer}")
print(f"Average Ratio: {min_ratio:.2f}")

print("nAll Layers' Average Ratios (sorted):")
print(layer_avg_ratios.sort_values())


In [ ]:
# Setting up main parameters
class_id = 1 #1/2
sample_id = 11 #data number in the dataset
n_concepts =  3#jj
n_refimgs = 12 # constant
layer = "module.backbone.ERBlock_4.1.block.1.rbr_1x1.conv" # change the layer
mode = "relevance"
prediction_num = 0 

# if failing, try to restart the notebook and do not run analysis again, go directly to plotting
plot_explanations(model_name, model, dataset, sample_id, class_id, layer, prediction_num, mode, n_concepts, n_refimgs, output_dir=output_dir)

In [ ]:
import torch
import copy
import logging
import random
import pandas as pd
import re

# === Setup ===
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

n_samples = 100
dataset_size = len(dataset)
n_concepts = 3
class_id = 1
model_name = "yolov6s6"

yolo_layers = [
    "module.backbone.ERBlock_4.1.block.1.rbr_1x1.conv",
]

log_file_path = "../examples/log_yolo_test_4.1.txt"

# === Configure Logging ===
logging.basicConfig(filename=log_file_path, level=logging.ERROR, force=True)

# === Attribution Loop ===
sample_indices = random.sample(range(dataset_size), n_samples)

for i, sample_idx in enumerate(sample_indices):
    layer_idx = i % len(yolo_layers)
    layer = yolo_layers[layer_idx]

    image_tensor, _ = dataset[sample_idx]
    image_tensor = image_tensor.to(device)  #  Move to same device as model

    # Log sample & layer before attribution
    logging.error(f"SAMPLE {sample_idx}, LAYER {layer}")

    attribution = ATTRIBUTORS[model_name](model)
    composite = COMPOSITES[model_name](canonizers=[CANONIZERS[model_name]()])
    condition = [{"y": class_id}]

    input_tensor = copy.deepcopy(image_tensor).unsqueeze(0).requires_grad_()

    #  Ensure input_tensor is on correct device (defensive check)
    input_tensor = input_tensor.to(device)

    attr = attribution(
        input_tensor,
        condition,
        composite,
        record_layer=[layer],
        init_rel=1
    )

print("Attribution complete, logs saved.")

# === Parsing Logs & Building Tables ===
with open(log_file_path, 'r') as f:
    logs = f.readlines()

results = []
sample_id = None
layer_name = None

for line in logs:
    sample_match = re.match(r'.*SAMPLE (\d+), LAYER (.+)', line)
    if sample_match:
        sample_id = int(sample_match.group(1))
        layer_name = sample_match.group(2)
    time_match = re.search(r'Prediction time: ([\d\.]+), Backward time: ([\d\.]+), Full attribution time: ([\d\.]+)', line)
    if time_match and sample_id is not None:
        results.append({
            "Sample ID": sample_id,
            "Layer": layer_name,
            "Prediction Time (s)": float(time_match.group(1)),
            "Backward Time (s)": float(time_match.group(2)),
            "Full Attribution Time (s)": float(time_match.group(3))
        })
        sample_id, layer_name = None, None

# === Save Detailed Sample Results ===
df = pd.DataFrame(results)
df = df.sort_values(by="Sample ID")
df.to_csv("detailed_sample_times_4.1.csv", index=False)

# === Compute & Save Global Summary Table ===
avg_pred_time = df["Prediction Time (s)"].mean()
global_lcrp_time = df["Full Attribution Time (s)"].sum()
attr_single = df["Backward Time (s)"].mean()
allowed_time = attr_single * dataset_size

summary_table = pd.DataFrame({
    "Metric": [
        "Model Prediction (average per sample)",
        "Global (LCRP) Total Time",
        "Attribution Time per Sample",
        "Allowed Time for Global Explanation",
        "Number of Data Points"
    ],
    "Value": [
        f"{avg_pred_time:.4f} s",
        f"{global_lcrp_time:.2f} s",
        f"{attr_single:.4f} s",
        f"{allowed_time:.2f} s",
        f"{dataset_size}"
    ]
})

summary_table.to_csv("KPI_explanation_summary_4.1.csv", index=False)

# === Display Output ===
print("\n==== Global Explanation Summary ====")
print(summary_table.to_string(index=False))
print("Results saved to 'detailed_sample_times.csv' and 'KPI_explanation_summary_4.1.csv'")


In [ ]:
import pandas as pd

# Load the detailed sample times (produced by the main script)
df = pd.read_csv("detailed_sample_times_4.1.csv")

# Calculate the Backward / Prediction Time Ratio per sample
df["Backward/Prediction Ratio"] = df["Backward Time (s)"] / df["Prediction Time (s)"]
avg_ratio = df["Backward/Prediction Ratio"].mean()

print(f"Average Backward Time / Prediction Time Ratio: {avg_ratio:.2f}")

# Load summary table to append the new metric
summary_table = pd.read_csv("global_explanation_summary.csv")

# Append the ratio to the summary table
summary_table = pd.concat([
    summary_table,
    pd.DataFrame({
        "Metric": ["4_1 Average Backward/Prediction Time Ratio"],
        "Value": [f"{avg_ratio:.2f}"]
    })
], ignore_index=True)

# Save updated summary
summary_table.to_csv("global_explanation_summary_4.1.csv", index=False)

print("Summary table updated with ratio.")


In [16]:
!pkill -u heydari -f jupyter